# Introduction
---

# Vorgehensweise in dieser Arbeit

Die Arbeit orientiert sich am Data Science Lifecycle, welcher nachfolgend dargestellt wird.

Die blau markierte Phase der Planung wird zur Vollständigkeit auf dieser Seite in einem rudimentären Niveau durchgegangen.</br>

Das Feature Engineering wird gleichzeitig auf Test- und Trainingsdaten angewendet, wenn sichergestellt ist das keine Informationen des jeweils anderen Datasets in die Modelle einfließen. Beispielsweise kann eine Wurzel-Transformation sofort erfolgen, weil kein Mittelwert oder sonstiger Lagewert verwendet wird. Im Gegensatz dazu wird eine Standardisierung, welche den Mittelwert verwendet, bei Bedarf separat auf das jeweilige Datensets angewendet.

<img src="https://kirenz.github.io/ds-python/_images/lifecycle.png" alt="alt text" width="600" align="center">

(Kirenz, J. 2021: https://kirenz.github.io/ds-python/docs/lifecycle.html)

## Planing

Zur Bewertung von Immobilienpreisen in bestimmten Distrikten sollen mehrere Modelle erstellt werden.

Geschätzt werden soll der Median-Preis von Immobilien mit folgenden Modellen.

* OLS-Regression
* Lasso-Regression
* Regression mit Splines

Ferner wird klassifiziert, ob die Median-Preise in Distrikten über der Preisschwelle von 150T$ liegen.

* Logistische Regression

## Variablen

Der zur Verfügung gestellte Datensatz enthält folgende Werte.

---
__longitude und latidue__: Koordinaten zur Orientierung</br>
__housing_median_age:__ Median age of a house within a district; a lower number is a newer building</br>
__total_rooms:__ Total number of rooms within a district </br>
__total_bedrooms:__ Total number of bedrooms within a district</br>
__population:__ Total number of people residing within a district</br>
__households:__ Total number of households, a group of people residing within a home unit, for a district</br>
__median_income:__ Median income for households within a district of houses (measured in tens of thousands of US Dollars)</br>
__median_house_value:__ Median house value within a district (measured in US Dollars)</br>
__ocean_proximity:__ Location of the district</br>
__price_category:__ Indicator variable made from median_house_value (if median house value is below or above 150000)</br>

---

Mit Hilfe der nachfolgend dargestellten Variablen werden die Modelle trainiert und getestet.
In den Modellen werden die Variabelen *price_categorie* und *median_house_value* die Responsewerte darstellen.</br>
Ferner liegt mit *ocean_proximity* eine weitere Variable zur Strand- oder Ozean-Nähe vor.</br>
Die restlichen Variablen sind absolute oder Medianwerte.

In [1]:
import pandas as pd
df = pd.read_csv("project_data.csv")

In [9]:
df.groupby(by=["price_category","ocean_proximity"]).describe().T

price_category               above                                           \
ocean_proximity          <1H OCEAN        INLAND       ISLAND      NEAR BAY   
longitude      count   7938.000000   4517.000000     5.000000   1997.000000   
               mean    -118.868776   -119.744651  -118.354000   -122.263450   
               std        1.586794      1.914385     0.070569      0.153829   
               min     -124.140000   -123.730000  -118.480000   -122.590000   
               25%     -118.530000   -121.370000  -118.330000   -122.420000   
               50%     -118.290000   -119.850000  -118.320000   -122.260000   
               75%     -117.990000   -118.030000  -118.320000   -122.130000   
               max     -116.620000   -114.310000  -118.320000   -122.010000   
latitude       count   7938.000000   4517.000000     5.000000   1997.000000   
               mean      34.578614     36.741508    33.358000     37.786900   
               std        1.459029      2.110433     0.040866      0.185200   
               min       32.610000     32.640000    33.330000     37.350000   
               25%       33.850000     34.420000    33.340000     37.710000   
               50%       34.040000     36.810000    33.340000     37.780000   
               75%       34.230000     38.530000    33.350000     37.890000   
               max       41.680000     41.950000    33.430000     38.340000   
total_rooms    count   7938.000000   4517.000000     5.000000   1997.000000   
               mean    2721.910809   2561.917866  1574.600000   2567.541813   
               std     2195.687026   2310.304028   707.545264   1877.217757   
               min       11.000000      6.000000   716.000000      8.000000   
               25%     1521.000000   1343.000000   996.000000   1472.000000   
               50%     2197.500000   2033.000000  1675.000000   2156.000000   
               75%     3244.000000   2992.000000  2127.000000   3114.000000   
               max    37937.000000  39320.000000  2359.000000  18634.000000   
total_bedrooms count   7850.000000   4473.000000     5.000000   1977.000000   
               mean     554.553248    503.413146   420.400000    521.805766   
               std      430.359921    420.666301   169.320111    371.283369   
               min        5.000000      2.000000   214.000000      1.000000   
               25%      307.000000    275.000000   264.000000    290.000000   
               50%      444.000000    406.000000   512.000000    427.000000   
               75%      661.000000    599.000000   521.000000    648.000000   
               max     5471.000000   6210.000000   591.000000   3226.000000   
population     count   7938.000000   4517.000000     5.000000   1997.000000   
               mean    1507.103930   1322.233341   668.000000   1239.056585   
               std     1106.595859   1088.419749   301.691067    897.894411   
               min        3.000000      5.000000   341.000000      8.000000   
               25%      850.000000    703.000000   422.000000    718.000000   
               50%     1236.000000   1090.000000   733.000000   1039.000000   
               75%     1834.000000   1606.000000   744.000000   1500.000000   
               max    16122.000000  16305.000000  1100.000000   8276.000000   
households     count   7938.000000   4517.000000     5.000000   1997.000000   
               mean     526.294659    449.503210   276.600000    496.882824   
               std      393.965095    364.584243   113.200265    355.133832   
               min        4.000000      2.000000   160.000000      1.000000   
               25%      297.000000    245.000000   173.000000    279.000000   
               50%      427.000000    371.000000   288.000000    410.000000   
               75%      626.000000    542.000000   331.000000    616.000000   
               max     5189.000000   5358.000000   431.000000   3589.000000   
median_income  count   7938.000000   4517.000000  